# Statistical Arbitrage Final Project - Gaussian Trading Method

# Data preprocessing

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('BTC_data.csv', index_col=0, parse_dates=True)
df['return'] = df['Close'].pct_change()
df.dropna(inplace=True)

In [4]:
# Divide the data into training and testing set using 80% of the data for training
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]
train_return = train['return']
test_return = test['return']

In [5]:
from sklearn.gaussian_process.kernels import Kernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
class FractionalBrownianMotionKernel(Kernel):
    def __init__(self, H):
        self.H = H

    def __call__(self, X, Y=None):
        X = np.atleast_2d(X)
        two_H = 2 * self.H
        Y = X if Y is None else np.atleast_2d(Y)
        cov = 0.5 * (np.abs(X) ** two_H + np.abs(Y.T) ** two_H - np.abs(X - Y.T) ** two_H)
        return cov

    def diag(self, X):
        return np.ones(X.shape[0])

    def is_stationary(self):
        return True
    
class OUProcessKernel(Kernel):
    def __init__(self, alpha, sigma):
        self.alpha = max(alpha, 1e-4)
        self.sigma = sigma
    
    def __call__(self, X, Y=None):
        X = np.atleast_2d(X)
        Y = X if Y is None else np.atleast_2d(Y)
        coeff = self.sigma**2 / 2 / self.alpha
        cov = coeff * (np.exp(-self.alpha * np.abs(X - Y.T)) - np.exp(-self.alpha * (X + Y.T)))
        return cov

    def diag(self, X):
        return np.ones(X.shape[0])

    def is_stationary(self):
        return True

In [6]:
def fit_log_marginal_likelihood(x, kernel: Kernel):
    _x = x.reshape(-1, 1)
    _t = np.arange(len(_x))/252
    _t = _t.reshape(-1, 1)
    gp = GaussianProcessRegressor(kernel=kernel)
    gp.fit(_t, _x)
    return gp.log_marginal_likelihood_value_

def fit_predict_unit_distance_data(x, kernel: Kernel):
    _x = x.reshape(-1, 1)
    _t = np.arange(len(_x))/252
    _t = _t.reshape(-1, 1)
    gp = GaussianProcessRegressor(kernel=kernel)
    gp.fit(_t, _x)
    y_pred, sigma = gp.predict(np.array([_t[-1] + 1 /252]), return_std=True)
    return y_pred[0], sigma[0]


# expand grid by params
def expand_grid(params):
    import itertools
    keys, values = zip(*params.items())
    grid = pd.DataFrame(list(itertools.product(*values)), columns=keys)
    return grid    

def optimal_kernel_params(params, kernel_class, x):
    grid = expand_grid(params)
    log_marginal_likelihoods = []
    for i in range(len(grid)):
        kernel = kernel_class(**grid.iloc[i].to_dict())
        log_marginal_likelihoods.append(fit_log_marginal_likelihood(x, kernel))
    grid['log_marginal_likelihood'] = log_marginal_likelihoods
    grid_max = grid.loc[grid['log_marginal_likelihood'].idxmax()]
    return grid_max

In [7]:

rolling_window = 250

temp_train = train_return.iloc[:rolling_window].copy()
temp_y = temp_train.values

temp_kernel = OUProcessKernel(alpha=0.2, sigma=0.4)
y_pred, sigma = fit_predict_unit_distance_data(temp_y, temp_kernel)
print(y_pred, sigma)

-0.01593652100264411 0.9325314944337306


In [8]:

params = {
    'alpha': np.linspace(0.1, 1.0, 10),
    'sigma': np.linspace(0.1, 1.0, 10)
}

grid = expand_grid(params)
grid

,alpha,sigma
0,0.1,0.1
1,0.1,0.2
2,0.1,0.3
3,0.1,0.4
4,0.1,0.5
...,...,...
95,1.0,0.6
96,1.0,0.7
97,1.0,0.8
98,1.0,0.9


In [9]:
# For each parameter combination, calculate the log marginal likelihood
log_marginal_likelihoods = []
for i in range(len(grid)):
    temp_kernel = OUProcessKernel(alpha=grid['alpha'][i], sigma=grid['sigma'][i])
    # print('Fitting model with alpha =', grid['alpha'][i], 'sigma =', grid['sigma'][i])
    log_marginal_likelihoods.append(fit_log_marginal_likelihood(temp_y, temp_kernel))
    # print('Log marginal likelihood:', log_marginal_likelihoods[-1])

grid['log_marginal_likelihood'] = log_marginal_likelihoods

In [10]:
grid_max = grid.loc[grid['log_marginal_likelihood'].idxmax()]
print('Best parameters:', grid_max)

Best parameters: alpha                      1.000000e+00
sigma                      9.000000e-01
log_marginal_likelihood   -2.586606e+07
Name: 98, dtype: float64


In [11]:
optimal_kernel_params(params, OUProcessKernel, temp_y)

alpha                      1.000000e+00
sigma                      9.000000e-01
log_marginal_likelihood   -2.586606e+07
Name: 98, dtype: float64

In [12]:
params = {
    'H': np.linspace(0.1, 1.0, 10)
}
optimal_kernel_param = optimal_kernel_params(params, FractionalBrownianMotionKernel, temp_y)
optimal_kernel_param_dict = optimal_kernel_param.to_dict()
optimal_kernel_param_dict.pop('log_marginal_likelihood')
optimal_kernel = FractionalBrownianMotionKernel(**optimal_kernel_param_dict)
fit_predict_unit_distance_data(temp_y, optimal_kernel)

(-0.01594917732036063, 0.10910894557625055)

In [13]:
temp_y

array([-7.19255760e-02, -6.98426451e-02,  3.57349164e-02, -2.46585445e-02,
        8.35210165e-03,  8.36474178e-02, -2.88808143e-02, -2.74830917e-02,
       -1.73699438e-02, -1.21283277e-02, -5.59145722e-02, -4.54420890e-03,
        3.05672387e-02, -8.60333797e-03, -2.22697222e-02, -4.14853626e-02,
       -8.52433205e-02, -2.54084874e-02,  2.98555426e-02,  1.85046656e-02,
        4.98323556e-02,  3.42437764e-02, -9.48970552e-03,  2.03837651e-03,
        4.48524547e-02,  3.13433397e-02,  2.67818101e-02, -1.52093819e-02,
       -3.09469244e-02,  3.14201280e-03,  2.00230108e-02, -4.84361965e-03,
       -1.72020479e-02,  9.48165673e-03, -8.58274545e-03, -6.45712682e-02,
       -2.00858130e-04, -3.09031941e-02,  2.14040735e-02, -4.83500691e-03,
        1.31137164e-02, -6.15936764e-02,  2.89459168e-02, -2.02255352e-02,
       -3.71600372e-02,  4.39004799e-04,  5.09981010e-03,  8.96957121e-03,
        2.72139353e-02,  2.88790004e-02, -1.96827846e-02,  8.97448409e-03,
        5.14518612e-02,  

In [15]:
params = {
    'length_scale': np.linspace(0.01, 1.0, 20)
}

optimal_kernel_param = optimal_kernel_params(params, RBF, temp_y)
print(optimal_kernel_param)

optimal_kernel_param_dict = optimal_kernel_param.to_dict()
optimal_kernel_param_dict.pop('log_marginal_likelihood')
optimal_kernel = RBF(**optimal_kernel_param_dict)
fit_predict_unit_distance_data(temp_y, optimal_kernel)


/Users/jamesguan/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/jamesguan/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/jamesguan/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/jamesguan/anaconda3/lib/python3.11/site-packages/sklea

length_scale                 0.010000
log_marginal_likelihood   -229.924323
Name: 0, dtype: float64


/Users/jamesguan/anaconda3/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


(0.0, 1.0)